In [290]:
using Pkg
Pkg.add(["JuMP", "GLPK", "IterTools", "Combinatorics", "ProgressMeter", "Graphs", "GraphPlot", "GameTheory", "DataFrames", "CSV"])
# Pkg.add(["Ipopt"])

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.6/Project.toml`
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [1]:
using JuMP, GLPK, IterTools, Combinatorics, ProgressMeter, Graphs, GraphPlot, DataFrames, CSV

In [117]:
case_name = "mock_2p"

V = [:S, :D]

E_f = [
    (:ef1, :S, :D, 0.5, 20.)   
]

E_t = [
    (:et1, :S, :D, 0.2, 15.),
    (:et2, :S, :D, 0.2, 20.),
]

F = 100.

toll_range = 0:2:20

;

In [104]:
case_name = "mock_4p"

V = [:S, :A, :B, :C, :D]

E_f = [
    (:ef1, :S, :A, 0.5, 20.),
    (:ef2, :A, :B, 0.5, 20.),
    (:ef3, :B, :C, 0.5, 20.),
    (:ef4, :C, :D, 0.5, 20.),
]

E_t = [
    (:et1, :S, :A, 0.5, 15.),
    (:et2, :S, :C, 1.4, 50.),
    (:et3, :B, :D, 0.9, 30.),
    (:et4, :C, :D, 0.4, 20.)
]

F = 100.

toll_range = 0:2:20

;

In [118]:
E = Dict([(e[1], e) for e in vcat(E_f, E_t)])

G_in = Dict()
G_out = Dict()
for (k, v) in E
    ename = k
    esrc = v[2]
    edest = v[3]
    if !(esrc in keys(G_out))
        G_out[esrc] = []
    end
    push!(G_out[esrc], ename)
    if !(edest in keys(G_in))
        G_in[edest] = []
    end
    push!(G_in[edest], ename)
end

In [119]:
# V_idx = Dict([(V[i], i) for i in 1:length(V)])
# graph = Graphs.()
# add_vertices!(graph, length(V))
# for (_, e) in E
#     add_edge!(graph, (V_idx[e[2]], V_idx[e[3]]))
# end
# gplot(graph, edgelabel=[e[1] for e in values(E)], nodelabel=V)

In [120]:
function route(prev, curr)
    if curr == :D
        return [prev]
    end
    paths = []
    for e in G_out[curr]
        curr_path = vcat(prev, e)
        next_nd = E[e][3]
        for p in route(curr_path, next_nd)
            push!(paths, p)
        end
    end
    return paths
end

route (generic function with 1 method)

In [121]:
R = route([], :S)

3-element Vector{Any}:
 Any[:ef1]
 Any[:et2]
 Any[:et1]

In [122]:
M = 1e4
m = 1e-4
;

In [123]:
function travel_equilibrium(toll_prices)

    model = Model(GLPK.Optimizer)
    @variable(model, x[keys(E)])
    @variable(model, y[1:length(R)])
    @variable(model, h[1:length(R)], Bin)
    @variable(model, r)

    @constraint(model, x .>= 0)
    @constraint(model, y .>= 0)
    @constraint(model, r .>= 0)
    
    @constraint(model, sum(y) == F)
    
    for nd in V
        if nd == :S
            @constraint(model, sum([x[i] for i in G_out[nd]]) == F)
        elseif nd == :D
            @constraint(model, sum([x[i] for i in G_in[nd]]) == F)
        else
            @constraint(model, sum([x[i] for i in G_in[nd]]) == sum([x[i] for i in G_out[nd]]))
        end
    end

    for (e, _) in E
        @constraint(model, sum([y[i] for i in 1:length(R) if e in R[i]]) == x[e])
    end

    for i in 1:length(R)
        @constraint(model, M * (h[i] - 1) + sum([E[e][4] * x[e] + E[e][5] + get(toll_prices, e, 0.) for e in R[i]]) <= r)
        @constraint(model, sum([E[e][4] * x[e] + E[e][5] + get(toll_prices, e, 0.) for e in R[i]]) >= r)
        @constraint(model, y[i] <= M * h[i])
    end
    @objective(model, Min, r)
    optimize!(model)
    
    return x, y, h, r
end

travel_equilibrium (generic function with 1 method)

In [124]:
function toll_revenue(x, toll_price)
    return Dict([(e, value(x[e] * get(toll_price, e, 0.))) for (e, _) in E])
end

function path_cost(h, toll_price)
    return [value(h[i] * sum([E[e][4] * x[e] + E[e][5] + get(toll_price, e, 0.) for e in R[i]])) for i in 1:length(R)]
end

path_cost (generic function with 1 method)

In [125]:
results = Dict()

@showprogress for tp in product([[(e[1], t) for t in toll_range] for e in E_t]...)
    results[tp] = travel_equilibrium(Dict(tp))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [126]:
results_revenue = Dict()

@showprogress for (tp, (x, y, h, r)) in results
    results_revenue[tp] = toll_revenue(x, Dict(tp))
end

In [127]:
# for (tp, (x, y, h, r)) in results
#     @show tp
#     toll_prices = Dict(tp)
#     flow = [value(x[e[1]]) for e in E_t]
#     @show flow
#     for (i, rt) in enumerate(R)
#         println(rt, " ", value(h[i] * sum([E[e][4] * x[e] + E[e][5] + get(toll_prices, e, 0.) for e in rt])))
#     end
#     println()
# end

In [128]:
data_rows = []

@showprogress for (tp, (x, y, h, r)) in results
    rev = results_revenue[tp]
    data_row = Dict()
    for (p, t) in tp
        data_row[Symbol(join([string(p), "_tollprice"]))] = t
        data_row[Symbol(join([string(p), "_revenue"]))] = rev[p]
    end
    push!(data_rows, NamedTuple{Tuple(keys(data_row))}(values(data_row)))
end

In [129]:
CSV.write("toll_equil_$(case_name).csv", DataFrame(data_rows))

"toll_equil_mock_2p.csv"

In [136]:
value.(results[((:et1, 14), (:et2, 10))][1])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:ef1, :et2, :et1]
And data, a 3-element Vector{Float64}:
 32.5
 31.25
 36.25

In [67]:
# function construct_game(coalitions)
#     players = [p for c in coalitions for p in c]
#     total_players = length(players)
#     player_idx = Dict([(p, i) for (i, p) in enumerate(players)])
#     player_coalition = Dict([(p, i) for (i, c) in enumerate(coalitions) for p in c])
#     total_prices = length(toll_range)
#     gains = Array{Float64}(undef, [total_prices for _ in players]..., total_players)
#     for c_choices_idx in product([1:total_prices for _ in players]...)
#         c_choices = [(players[c_idx], toll_range[i]) for (c_idx, i) in enumerate(c_choices_idx)]
#         c_choices = Tuple(sort(c_choices))
#         rev_indiv = results_revenue[c_choices]
#         rev_coal = [sum([rev_indiv[e] for e in c]) for c in coalitions]
#         for p in 1:total_players
#             gains[c_choices_idx..., p] = rev_coal[player_coalition[players[p]]]
#         end
#     end
#     game = GameTheory.NormalFormGame(gains)
#     return (game, player_idx)
# end

In [66]:
# function sequential_best_response(g::NormalFormGame{N};
#                                   init_actions::Vector{Int}=ones(Int, N),
#                                   tie_breaking=:smallest,
#                                   verbose=false) where N
#     a = copy(init_actions)
#     if verbose
#         println("init_actions: $a")
#     end
    
#     new_a = Array{Int}(undef, N)
#     max_iter = prod(g.nums_actions)
    
#     for t in 1:max_iter
#         copyto!(new_a, a)
#         for (i, player) in enumerate(g.players)
#             if N == 2
#                 a_except_i = new_a[3-i]
#             else
#                 a_except_i = (new_a[i+1:N]..., new_a[1:i-1]...)
#             end
#             new_a[i] = best_response(player, a_except_i,
#                                      tie_breaking=tie_breaking)
#             if verbose
#                 println("player $i: $new_a")
#             end
#         end
#         if new_a == a
#             return a
#         else
#             copyto!(a, new_a)
#         end
#     end
    
#     println("No pure Nash equilibrium found")
#     return a
# end

In [ ]:
# function get_equilbrium(coalition)
#     game, player_idx = construct_game(coalition)
#     eq = sequential_best_response(game)

In [64]:
# function get_coalition(all_party, big_coalition)
#     coalitions = [big_coalition]
#     for p in all_party
#         if !(p in big_coalition)
#             push!(coalitions, [p])
#         end
#     end
#     return coalitions
# end

# prog = Progress(2^length(toll_edges) - 1)
# toll_edges = sort([e[1] for e in E_t])
# powerset_score = Dict()
# for big_coalition in powerset(toll_edges)
#     if length(big_coalition) > 0
#         ProgressMeter.next!(prog)
#         big_coalition = sort(Array(big_coalition))
#         coalitions = get_coalition(toll_edges, big_coalition)
#         equil_tp = get_equilibrium(coalitions)
#         powerset_score[big_coalition] = coalition_revenue(coalitions, equil_tp)[1]
#     end
# end
# ProgressMeter.finish!(prog)

# powerset_score

In [65]:
# shapley = Dict((e, 0.) for e in toll_edges)

# for es_perm in permutations(toll_edges)
#     curr_gain = 0.
#     for i in 1:length(es_perm)
#         e_i = es_perm[i]
#         val = powerset_score[sort(es_perm[1:i])]
#         shapley[e_i] += (val - curr_gain)
#         curr_gain = val
#     end
# end

# multiplier = powerset_score[toll_edges] / sum(values(shapley))
# for e in keys(shapley)
#     shapley[e] *= multiplier
# end

# shapley

In [350]:
value.(first(results)[2][2])

8-element Vector{Float64}:
  0.0
  0.0
 32.962318064112814
 13.500758996338838
  0.0
  8.869095454951477
  9.56246093401213
 35.10536655058475